In [1]:
from new_loader import TorchTensorFolderDataset
from torch.utils.data import Dataset, DataLoader
import os
def get_loaders():
    train_path = os.path.join(os.getcwd(),"data\\preprocessed\\raw\\train")
    val_path = os.path.join(os.getcwd(),"data\\preprocessed\\raw\\validation")
    test_path = os.path.join(os.getcwd(),"data\\preprocessed\\raw\\test")
    
    train_dataset = TorchTensorFolderDataset(train_path)
    val_dataset = TorchTensorFolderDataset(val_path)
    test_dataset = TorchTensorFolderDataset(test_path)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size = 16)
    val_loader = DataLoader(val_dataset, shuffle=False, batch_size = 16)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size = 16)

    return {"train":train_loader, "val":val_loader, "test":test_loader}

def get_denoised_loaders():
    
    train_path = os.path.join(os.getcwd(),"data\\preprocessed\\denoised\\train")
    val_path = os.path.join(os.getcwd(),"data\\preprocessed\\denoised\\validation")
    test_path = os.path.join(os.getcwd(),"data\\preprocessed\\denoised\\test")
    
    train_dataset = TorchTensorFolderDataset(train_path)
    val_dataset = TorchTensorFolderDataset(val_path)
    test_dataset = TorchTensorFolderDataset(test_path)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size = 16)
    val_loader = DataLoader(val_dataset, shuffle=False, batch_size = 16)
    test_loader = DataLoader(test_dataset, shuffle=False, batch_size = 16)

    return {"train":train_loader, "val":val_loader, "test":test_loader}

In [2]:
from CNN_transfomers import Mel_transformer, RawAudioTransformer, train_transformer, RefinedRawAudioTransformer
loaders = get_loaders()
train_loader, val_loader = loaders["train"], loaders["val"]

In [4]:
spect_model = Mel_transformer(num_classes=30)
spect_model = train_transformer(spect_model, train_loader, val_loader, epochs=20)


KeyboardInterrupt: 

In [4]:
from torchinfo import summary
summary(spect_model)

Layer (type:depth-idx)                                            Param #
Mel_transformer                                                   25,856
├─MelSpectrogram: 1-1                                             --
│    └─Spectrogram: 2-1                                           --
│    └─MelScale: 2-2                                              --
├─AmplitudeToDB: 1-2                                              --
├─Sequential: 1-3                                                 --
│    └─Conv2d: 2-3                                                320
│    └─BatchNorm2d: 2-4                                           64
│    └─ReLU: 2-5                                                  --
│    └─Conv2d: 2-6                                                18,496
│    └─BatchNorm2d: 2-7                                           128
│    └─ReLU: 2-8                                                  --
├─Linear: 1-4                                                     1,048,832
├─Transforme

In [ ]:
raw_model = RawAudioTransformer(num_classes=30)

raw_model = train_transformer(raw_model, train_loader, val_loader, epochs=20)


Epoch 1/20:  21%|██        | 664/3193 [02:17<08:50,  4.77it/s, loss=2.62]

In [3]:
raw_model = RefinedRawAudioTransformer(num_classes=30)

raw_model = train_transformer(raw_model, train_loader, val_loader, epochs=20)

Epoch 1: Train Loss: 2.6728 | Train Acc: 0.2010 | Val Acc: 0.3236


Epoch 2: Train Loss: 1.9319 | Train Acc: 0.4013 | Val Acc: 0.4838


Epoch 3: Train Loss: 1.5371 | Train Acc: 0.5301 | Val Acc: 0.5981


Epoch 4: Train Loss: 1.2652 | Train Acc: 0.6156 | Val Acc: 0.6624


Epoch 5: Train Loss: 1.0852 | Train Acc: 0.6697 | Val Acc: 0.6961


Epoch 6: Train Loss: 0.9471 | Train Acc: 0.7116 | Val Acc: 0.6945


Epoch 7: Train Loss: 0.8554 | Train Acc: 0.7388 | Val Acc: 0.7390


Epoch 8: Train Loss: 0.7691 | Train Acc: 0.7673 | Val Acc: 0.7530


Epoch 9: Train Loss: 0.7053 | Train Acc: 0.7846 | Val Acc: 0.7773


KeyboardInterrupt: 

In [2]:
from CNN_transfomers import Mel_transformer, RawAudioTransformer, train_transformer, RefinedRawAudioTransformer
loaders = get_denoised_loaders()
train_loader, val_loader = loaders["train"], loaders["val"]

In [3]:
raw_model = RawAudioTransformer(num_classes=30)

raw_model = train_transformer(raw_model, train_loader, val_loader, epochs=20)


KeyboardInterrupt: 

In [ ]:
spect_model = Mel_transformer(num_classes=30)
spect_model = train_transformer(spect_model, train_loader, val_loader, epochs=20)
